Check SPA for pictures of people related to Johan Emanuel Wikström
* this [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20Johan%20Emanuel%20Wikström.ipynb) 

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-10-23 02:52:04.812209


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php" 

In [13]:
from IPython.display import HTML, Image
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

queryJohan = """PREFIX gas: <http://www.bigdata.com/rdf/gas#>

SELECT distinct (CONCAT (?itemLabel," ",str(year(?born))) AS ?search) ?item ?itemLabel ?born  
 (str(year(?born)) AS ?year) {
  SERVICE gas:service {
    gas:program gas:gasClass "com.bigdata.rdf.graph.analytics.SSSP" ;
                gas:in wd:Q20720200 ;
                gas:traversalDirection "Undirected" ;
                gas:out ?item ;
                gas:out1 ?depth ;
                gas:maxIterations 4 ;
                gas:linkType wdt:P40 .
  }
  ?item wdt:P27 wd:Q34
  OPTIONAL { ?item wdt:P569 ?born }
  OPTIONAL { ?item wdt:P40 ?linkTo }
  OPTIONAL { ?item wdt:P18 ?pic }
  SERVICE wikibase:label {bd:serviceParam wikibase:language "sv,en" }
  FILTER ( YEAR(?born) < 1880) # born before 1880

} group by ?item ?itemLabel ?born
order by ?itemLabel """

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#SPAdf = get_sparql_dataframe(endpoint_url, query)
SPAdf = get_sparql_dataframe(endpoint_url, queryJohan)

SPAdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   search     28 non-null     object
 1   item       28 non-null     object
 2   itemLabel  28 non-null     object
 3   born       28 non-null     object
 4   year       28 non-null     object
dtypes: object(5)
memory usage: 1.2+ KB


In [28]:
SPAdetail = "https://portrattarkiv.se/details/"
for index, row  in SPAdf.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "10",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []


    for h in data["hits"]["hits"]:
        id = h["_id"]
#        print(h)
        source = h["_source"]
        try:
#            print(id)
#           urlPicture = urlbasePic + id
#            urls.append(urlPicture)
            score = h["_score"]
            FirstName = source["FirstName"]
            LastName = source["LastName"]
            BirthYear = source["BirthYear"]
            print("\t\t",score,FirstName, " ", LastName, " - ", BirthYear,SPAdetail+id, "\t", )
        except:
            print("Error")



 Johanna Näslund 1838 http://www.wikidata.org/entity/Q108264383
		 15.370123 Anders   Almquist  -  1838 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA3smg 	
		 15.2288265 Gustaf   Askergren  -  1838 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA4JPA 	
		 15.2288265 Oskar   Appelberg  -  1838 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA38hA 	
		 15.065046 Hanna   Ouchterlony  -  1838 https://portrattarkiv.se/details/vfWYdYT6EoAAAAAAAAAvQg 	
		 14.95533 Alexander   Roth  -  1838 https://portrattarkiv.se/details/9Y5iReTPHdAAAAAAAAAkGw 	
		 14.79955 Ernst   Schoug  -  1838 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAABkpA 	
		 14.737883 Carl   Frisk  -  1838 https://portrattarkiv.se/details/IQojCnw0WmAAAAAAAAA1zg 	
		 14.72971 Johan   Hafström  -  1838 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAAFoA 	
		 14.724401 Christian   Bratt  -  1838 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA3pCQ 	
		 14.085737 Nils   Ödman  -  1838 https://portrattarkiv.se/d

		 21.263405 Frideborg   Danell-Beckman  -  1869 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZp6A 	
		 16.903456 Hedvig   Skarstedt  -  1869 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAACnPQ 	
		 16.520412 Hedvig   Skarstedt  -  1869 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAACnPg 	
		 14.765091 John   Tjerneld  -  1869 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABNq3w 	
		 14.738942 John   Tjerneld  -  1869 https://portrattarkiv.se/details/NhqpvvI9tpAAAAAAAAA1dg 	
		 10.175696 Ewert   Jonsson  -  1869 https://portrattarkiv.se/details/Di4a42b6X_AAAAAAAACIAw 	
		 9.577807 Elisabet   Tingvall  -  1869 https://portrattarkiv.se/details/NhqpvvI9tpAAAAAAAAAbDA 	
		 9.564344 Erik Natanael   Söderberg  -  1869 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACd0Fw 	
		 9.562848 Theodor   Andersson  -  1869 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA5x3g 	
		 9.523809 Dagny   Bergling  -  1869 https://portrattarkiv.se/details/TNQaWo324IAAAAAAAAC-9g 	

 Helena

		 17.810986 Johanna   Johansson  -  1859 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAgXw 	
		 17.702826 Johanna   Johansson  -  1859 https://portrattarkiv.se/details/Di4a42b6X_AAAAAAAAAoPQ 	
		 17.427217 Johanna   Magnell-Edén  -  1859 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZrgw 	
		 17.360882 Johanna   Andersson  -  1859 https://portrattarkiv.se/details/28K7pO-SRrAAAAAAAAC4Gw 	
		 17.08999 Johanna   Wetterström  -  1859 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAABt9w 	
		 16.894451 Johanna   Åkerblom  -  1859 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZtHA 	
		 16.8323 Johanna   Mårtensson  -  1859 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAAA1yg 	
		 16.404465 Johanna   Olsson  -  1859 https://portrattarkiv.se/details/vfWYdYT6EoAAAAAAAAA_3A 	
		 16.177128 Johanna   Nilsson  -  1859 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAAVZg 	
		 15.3229065 Johanna   Svensson  -  1859 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAAAPNA 	

 J

		 19.596542 Per   Magnusson  -  1852 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAAAH4g 	
		 19.256989 Per   Wistrand  -  1852 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAABStA 	
		 19.222755 Per   Eriksson  -  1852 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAAq4w 	
		 19.170305 P   Hillström  -  1852 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAABWlg 	
		 18.680765 Bernhard   Ericsson  -  1852 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAAarQ 	
		 17.37483 INGRID   Ersdotter  -  1852 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAATog 	
		 16.733227 Per   Lilliecreutz  -  1852 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADifw 	
		 16.70972 Carl   Spånberg  -  1852 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAA1wg 	
		 16.671888 Per   Lilliecreutz  -  1852 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADfmQ 	
		 16.66492 Axel   Carlström  -  1852 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA7TWw 	


In [32]:
from IPython.display import HTML, Image

def _src_from_data(data):
    """Base64 encodes image bytes for inclusion in an HTML img element"""
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

def gallery(images, row_height='auto'):
    """Shows a set of images in a gallery that flexes with the width of the notebook.
    
    Parameters
    ----------
    images: list of str or bytes
        URLs or bytes of images to display

    row_height: str
        CSS height value to assign to all images. Set to 'auto' by default to show images
        with their native dimensions. Set to a value like '250px' to make all rows
        in the gallery equal height.
    """
    figures = []
    for image in images:
        if isinstance(image, bytes):
            src = _src_from_data(image)
            caption = ''
        else:
            src = image
            caption = f'<figcaption style="font-size: 0.6em">{image}</figcaption>'
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{src}" style="height: {row_height}">
              {caption}
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

In [45]:
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="
for index, row  in SPAdf.iterrows():
    print("\n",row["search"],row["item"])
    encoded_body = json.dumps({
        "limit": "10",
        "from": "0",
        "birthyear":row["year"],
        "all":row["search"]
    })
    
    r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

    if r.status != 200:
        print(r.status)
        continue
    
    data = json.loads(r.data.decode('utf-8'),) 
    urls = []
    for h in data["hits"]["hits"]:
        id = h["_id"]
        source = h["_source"]
        urlPicture = urlbasePic + id
        urls.append(urlPicture)
    #print(urls)
    #gallery(urls, row_height='200px')


 Johanna Näslund 1838 http://www.wikidata.org/entity/Q108264383

 Anton Julius Winblad I 1828 http://www.wikidata.org/entity/Q20667111

 Anton Wahlberg 1878 http://www.wikidata.org/entity/Q20652937

 Antonette Kristina Winblad II 1854 http://www.wikidata.org/entity/Q84323221

 Brita Christina Öknaberg 1799 http://www.wikidata.org/entity/Q86513219

 Carl Gustaf Näslund 1825 http://www.wikidata.org/entity/Q20664308

 Catharina Maria Martinell 1773 http://www.wikidata.org/entity/Q63302217

 Elsa Maria Elisabet Näslund 1829 http://www.wikidata.org/entity/Q25905894

 Erik Johan Örbom 1724 http://www.wikidata.org/entity/Q34850319

 Frideborg Winblad 1869 http://www.wikidata.org/entity/Q5503563

 Helena Gustafsdotter Ruuth 1729 http://www.wikidata.org/entity/Q81086857

 Israel Israelsson Näslund 1796 http://www.wikidata.org/entity/Q20816949

 Israel Israelsson Näslund IV 1823 http://www.wikidata.org/entity/Q81559062

 Johan Christopher Ruuth I 1767 http://www.wikidata.org/entity/Q50346736

 

In [42]:
gallery(urls, row_height='100px')